<table style="width: 100%;" id="nb-header">
    <tr style="background-color: transparent;"><td>
        <img src="https://d8a-88.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 12pt;"><strong>Economic Models</strong>, Fall 2019<br>
            Dr. Eric Van Dusen</p></td></tr>
</table>

In [1]:
import pandas as pd
from datascience import *
from sympy import *
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
from sklearn.linear_model import LinearRegression
%matplotlib inline
plt.style.use("seaborn-muted")

## Acknowledgements: ##
We would like to thank Professor Raymond Hawkins for his Economics 100B Problem Set that served as the basis for this assignment.

# Project 1: Modeling Macroeconomic Variables using the Cobb-Douglas Production Function

The goal of this homework is to gain experience completing the following key steps in the data science pipeline:

1. Downloading and converting raw data files into a format Python can interact with
2. Identifying and visualizing overall trends in the data using a process called Exploratory Data Analysis (EDA)
3. Using the data to complete a problem of prediction

We hope that at the end of this project, you will see how the skills you have learned in Data 8 and this class can prepare you for dealing with real world datasets, and how you can use them to answer questions about the economy or the world.

## The question you will be answering today is the following: How can we apply the Cobb-Douglas Production Function to understand the different ways countries produce output or GDP?

# Part 1: Retrieving the data and simplifying the problem

In almost all cases in industry, you will have to find the dataset that can answer the question you are trying to solve. However, as we are introducing you to this process, we will be providing the link to such a dataset.

The dataset we will be using today is called "Penn World Table" (PWT) and was compiled by the University of Groningen. It catalogues information on relative levels of income, output, input and productivity for 182 countries between 1950 and 2017.

Your task:
1. Go to: https://www.rug.nl/ggdc/productivity/pwt/ and download the Excel datafile.
2. Open the excel file, select the "Data" sheet, then File > Save As > Select a file name and **change the file format to Comma Separated Values (.csv)**.
3. Upload this .csv file to the same folder as this Notebook. Normally it will be located in: econ-fa19 > wk6. You can access the Datahub file explorer by clicking the Jupyter logo on the top-left of the notebook. Uploading is accomplished by dragging the file into the file explorer window. You should see a blue button labeled "upload".
4. Edit the line ```file_name``` such that it matches the name of the .csv file.
5. Run the code cell below.

In [ ]:
### Edit the line below ###
file_name = ...

### Do not change the line below ###
data = Table().read_table(file_name, engine='python')

In [ ]:
### Checking whether the table was imported correctly ###
assert(data.num_rows == 12376 and data.num_cols == 52, "Please try re-uploading the dataset. You are missing columns or rows.")

If you pass the assert statements above, then you have correctly read in the data file. Nice! You have converted an online dataset into a datatable that you can explore and manipulate in Python! Now, this dataset is saved under the table called ```data```. Look at the first 5 lines below:

In [ ]:
data.show(5)

To get an idea of the dataset's geographic scope, let us find out what countries are included, and if they are spelt in interesting ways. This will be helpful for us later on in our analysis.

<div class="alert alert-info">
    
<h3> Question 1.1. </h3>
Create a table with one column called "Country", containing all of the countries that appear in the <code>data</code> table sorted in alphabetical order. Assign it to the variable <code>all_countries</code>.
</div>

In [2]:
all_countries = ...

In [3]:
### Checking all_countries
assert(all_countries.num_rows == 182, "Check your table methods!")
assert(all_countries.row(23)[0] < all_countries.row(55)[0], "Did you sort?")

<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-3-403a0a2f6872>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(all_countries.num_rows == 182, "Check your table methods!")


AttributeError: 'ellipsis' object has no attribute 'num_rows'

Take a look at the ```data``` table again. Notice that there are a lot of data columns in the table? Lucklily, we won't need most of them. As we will be using the Cobb-Douglas Production Function, think about what variables are needed in the equation, and which ones we can take from the table. This is an important part of the data science process: understanding the dataset that you are using. Most real-world datasets provide documentation listing the definitions and equations underlying each variable. 

PWT provides this and has identified three variables that will be helpful to us:
1. cn $\Rightarrow$ Capital Stock in millions of USD
2. cgdpe $\Rightarrow$ Expenditure-side Real GDP in millions of USD
3. emp $\Rightarrow$ Number of Persons employed in millions

<div class="alert alert-info">
    
<h3> Question 1.2. </h3>
In the cell below do the following: <br />
1. List the variables used in the Cobb-Douglas function <br />
2. Identify which variables are missing from the dataset <br />
3. Give a one sentence explanation of what each variable quantifies (i.e. K measures the amount of ...)
</div>

** Your answer here **

<div class="alert alert-info">
    
<h3> Question 1.3. </h3>
Remove all columns except for <code>cn</code>, <code>cgdpe</code>, <code>emp</code>, <code>country</code> and <code>year</code> from <code>data</code>. Ensure that <code>country</code> and <code>year</code> are the two left-most columns. Call the new table <code>cleaned_data</code> and display its first few rows. Rename the <code>cn</code> column to <code>Capital Stock</code>, <code>cgdpe</code> to <code>Real GDP</code> and <code>emp</code> to <code>Labor Force</code>. Do not be afraid if you see a lot of <code>nan</code> values - it just means there is no data for that country-year.
</div>

In [ ]:
### Your code here ###
cleaned_data = ...

cleaned_data.show(5)

Our goal will be to predict what these variables are for each of the countries that we will be examining. From these, we will be able to explore how output is produced in each of these countries. The question is, how can we use the Cobb-Douglas Production function to solve for the missing variables? An easy way would be to take the natural log of the equation, making it linear, providing us with ways to quantify $\alpha$ and $A$.

<div class="alert alert-info">

<h3>Question 1.4.</h3>
In the cell below, using LaTeX, take the natural log of the Cobb-Douglas Production Function and rewrite it as a function of one variable. Show all of your work. Full credit will not be given if you just display the final simplified equation. <br />
<i> Hint: $\ln (Y) - \ln (L) = \text{...}$. </i>
</div>

** Your LaTeX here **

# Part 2: Exploring the Data

Whenever we are dealing with a larget dataset like PWT, it is a good idea to see how the variables interact with each other. A common method, especially when dealing with economic data, is to generate a time series. This is a plot of some economic variable on the y-axis and time on the x-axis. We are going to do just that for our important Cobb-Douglas variables for different countries.

<div class="alert alert-info">
    
<h3>Question 2.1.</h3> 
Create a table of GDP, Capital Stock and Labor for the USA from 1990 to 2017. Do the same for China and two other countries of your choice. Use the middle cell to check if your countries of choice exist in the dataset. Make sure to use the country name exactly as it appears in the data table. Place all of this data in one table called <code>comparison_data</code>. Display the first few rows of the table. Additionally, set the <code>country_array</code> variable to an array of the countries you selected. <br />
<i>Hint: Look at the <a href="https://d8a-88.github.io/econ-fa19/python-reference.html"> Python Reference</a> for a table function you can use.</i>
</div>

In [ ]:
### Run this cell to see a table of all the countries in the dataset.###
### Select two countries and proceed to the lower code cell. ###
all_countries.show()

In [ ]:
### Your code here ###
country_array = make_array("United States", "China", ...)
comparison_data = ...

In [ ]:
assert(len(country_array) == 4, "Make sure you only select 2 countries other than the USA and China!")
assert(country_array.isin(all_countries), "Are your selected countries in the all_countries table? Are they spelt correctly?")

<div class="alert alert-info">
    
<h3>Question 2.2.</h3>
To help us in later questions, fill in the blanks in the <code>country_table_plotter</code> function below. Its inputs will be a table of the form <code>comparison_data</code> and the names of the two columns plotted, <code>columnX</code> and <code>columnY</code>. The <code>country_table_plotter</code> function will plot <code>columnX</code> versus <code>columnY</code> using data from <code>data_table</code> for all the countries contained in <code>countries</code>. <br />
<i> Hint: Look at the <a href="https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.plot.html"> Matplotlib Plotting Reference</a> for ideas of what to place in the blanks. </i>
</div>

In [ ]:
def country_table_plotter(data_table, countries, columnX, columnY):
    
    ### Your code here ###
    for country in ...:
        plt.plot(..., ..., label = country, linewidth = 1)
    
    ### Do not change the code below ###
    plt.legend()
    plt.xlabel(columnX)
    plt.ylabel(f"{columnY} (Logarithmic Scale)")
    plt.yscale("log")

<div class="alert alert-info">
    
<h3>Question 2.3.</h3>
Produce a plot of time and capital stock for the countries in your table. In the markdown cell below your plot, identify differences between the countries and discuss what surprised you.
</div>

In [ ]:
### Your code here ###
...

- United States:

- China:

- Your Country:

- Your Country:

<div class="alert alert-info">
    
<h3>Question 2.4.</h3>
Produce a similar plot, but this time of time and labor for the countries in your table. In the markdown cell below your plot, identify differences between the countries and discuss what surprised you.
</div>

In [ ]:
### Your code here ###
...

- United States:

- China:

- Your Country:

- Your Country:

<div class="alert alert-info">
    
<h3>Question 2.5.</h3>
Create a plot of time and GDP for the countries in your table. In the markdown cell below your plot, using your knowledge of the Cobb-Douglas Production Function, identify differences between the countries and discuss these in relation to your findings about each of the country's levels of capital stock and labor. Also note about how these have changed over time, if at all.
</div>

In [ ]:
### Your code here ###
...

- United States:

- China:

- Your Country:

- Your Country:

# Part 3: Prediction and Comparison

We are now going to provide numerical backing to your discussions of visual differencies between the nations above by predicting values for $\alpha$ and $A$ for each of the 4 countries that we are examining. To do this, we will revisit the equation that you derived in question 1.4. 

<div class="alert alert-info">
    
<h3>Question 3.1.</h3>
A key step in the original paper by Cobb and Douglas was that they converted the data into an index. This is important because the data is measured in different units. Do this for each country and each of the variables. Let the 2011 value for each country be the base year, i.e. 100. Place this in a new table called <code>indexed_data</code> together with <code>country</code> and <code>year</code> columns. Display the first few rows. <br />
<i>Hint: The formula for calculating an index is as follows:
$$
\dfrac{Q_{\text{current year}}}{Q_{\text{base year}}} \, \text{for some variable } Q
$$</i>
</div>

In [ ]:
base_values = comparison_data.where("year", 2011)
indexed_Ks = make_array()
indexed_Ls = make_array()
indexed_Ys = make_array()

### Your code here ###

for country in ...:
    country_base_values = base_values.where(..., ...)
    current_Ks = comparison_data.where(..., ...).column(...) / ...
    current_Ls = comparison_data.where(..., ...).column(...) / ...
    current_Ys = comparison_data.where(..., ...).column(...) / ...
    
    indexed_Ks = np.append(..., ...)
    indexed_Ls = np.append(..., ...)
    indexed_Ys = np.append(..., ...)
    
indexed_data = Table().with_columns("country", ...
                                   "year", ...
                                   "Indexed K", ...
                                   "Indexed L", ...
                                   "Indexed Y", ...
                                   )

indexed_data.show(5)

<div class="alert alert-info">
    
<h3>Question 3.2(a).</h3>
Using the equation you derived in part 1 and the <code>indexed_data</code> table, calculate the two log ratios that you need to perform linear regression. Place them in the table <code>log_ratios</code> with <code>country</code> and <code>year</code> columns in the code cell below and display the first few rows. Name the columns appropriately. The <code>log_ratios</code> table should have 4 columns.
</div>

In [ ]:
### Your code here ###
log_ratios = ...

log_ratios.show(5)

<div class="alert alert-info">
    
<h3>Question 3.2(b).</h3>
Using the <code>country_table_scatter</code> function, plot the log ratios for each country from the <code>log_ratios</code> table below.
</div>

In [ ]:
### Do not change the code below ###
def country_table_scatter(data_table, columnX, columnY):
    # First getting a list of all the countries in data_table
    country_list = data_table.group("country").column("country")
    
    # For each country, creating a plot of columnX vs. columnY
    for country in country_list:
        curr_data_table = data_table.where("country", country)
        curr_data_table.scatter(columnX, columnY)
        plt.title(country)

In [ ]:
### Your code here ###
...

Now, onto the actual linear regression part of this project.

We will be using Scikit-Learn's linear regression function to get the $\alpha$ and $A$ values for each country. Here is the link to the documentation of the <a href = "https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html"> function</a>: 

Let us break down how this function works by taking a look at the examples section of the documentation.
``` 
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
```
The ```LinearRegression()``` function creates a LinearRegression object that has many different functions that we will be using, mainly ```.fit()``` and attributes like ```.coef_``` and ```.intercept_```. The ```.fit(X, y)``` function fits a line using the ```X``` and ```y``` arrays passed into it.

```
reg.coef_
array([1., 2.])
reg.intercept_ 
3.0000...
reg.predict(np.array([[3, 5]]))
array([16.])
```

After we have fit a line to the data, we can then get the corresponding attributes that we need: the slope coefficient and intercept. The example above is in 3 dimensions, x_0, x_1 and y. Thus, there are two slope coefficients for x_0 and x_1. Our data is in 2 dimensions so we will only get one slope coefficient and intercept per country. Think about the relationship between $\alpha$ and $A$ with regards to the slope and intercept from a linear regression fit?

<div class="alert alert-info">

<h3>Question 3.3a.</h3>
Using Sklearn's Linear Regression function, fit the data taken from each country. Think about what should be passed into the <code>fit()</code> function. If you are confused about this, take a look at the sklearn <a href ="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.fit"> documentation</a>:           

</div>

In [ ]:
### Create arrays of the data we will need from each country. ###
### Think about which variable should be on each axis. ###

china_x = ...
china_y = ...
us_x = ...
us_y = ...
country1_x = ...
country1_y = ...
country2_x = ...
country2_y = ...


### Do not change the code below ###

model_china = LinearRegression().fit(china_x.reshape(-1, 1), china_y.reshape(-1, 1))
model_us = LinearRegression().fit(us_x.reshape(-1, 1), us_y.reshape(-1, 1))
model_country1 = LinearRegression().fit(country1_x.reshape(-1, 1), country1_y.reshape(-1, 1))
model_country2 = LinearRegression().fit(country2_x.reshape(-1, 1), country2_y.reshape(-1, 1))

<div class="alert alert-info">
    
<h3>Question 3.3b.</h3>
Now that we have fit the data of each country, we can then retrieve the slope and intercept of each fit. Using the equation you derived in question 1.3., fill in the blanks in the print statements below such that they display the $\alpha$ and $A$ values for each country. Note that you will need to transform at least one of the variables.
</div>

In [ ]:
print(f"China alpha value: {...}")
print(f"United States alpha value: {...}")
print(f"... alpha value: {...}")
print(f"... alpha value: {...}")

In [ ]:
print(f"China A value: {...}")
print(f"United States A value: {...}")
print(f"... A value: {...}")
print(f"... A value: {...}")

<div class="alert alert-info">
    
<h3>Question 3.4.</h3>
With reference to the $\alpha$ and $A$ values for each of the countries you have examined, what do they indicate about that country's ability to produce output as measured through GDP? How does the allocation of scare resources as inputs to production differ between them? How about the role of technology or research and development?
</div>

** Your answer here **

# Congratulations! You've completed the project!